In [2]:
from pathlib import Path
from typing import List

from os import path
import numpy as np
from skimage import io,util,filters
from skimage.transform import EuclideanTransform, warp
from scipy import ndimage

from pystackreg import StackReg
# from twophotonUtils import parse_unregistered_channels
# from twophotonUtils import find_most_likely_z_slice_using_CC, z_translate_and_pad
# from imageLoadingWidgets import LoadTimepointForInspection

from os import path
from glob import glob
from re import findall
from tqdm import tqdm

import stackview

# Extract the first ome.tiff file from every subfolder, load, then separate the two channels
def sort_by_slice(filename):
    z = findall('_(\d+).ome.tif',filename)[0]
    return int(z)


<>:24: SyntaxWarning: invalid escape sequence '\d'
<>:24: SyntaxWarning: invalid escape sequence '\d'
/var/folders/zp/5k1kvm9j0t19cvq8h628kr8h0000gn/T/ipykernel_66928/2183388896.py:24: SyntaxWarning: invalid escape sequence '\d'
  z = findall('_(\d+).ome.tif',filename)[0]


# 0. Get file paths

Step one: Load all the OME-TIFFs and re-save as multipage TIFFs for each time point in region
Will perform a StackReg on each stack just in case there is small movement.


In [35]:
dirname = '/Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Two photon/NMS/Palbo senescence pilot/09-29-2025 Ear palbo pilot/M1tail 3017 H2B FUCCI/Right ear/4. Day 4/R2'

subfolders = glob(path.join(dirname,'ZSeries*/'))

header_ome_h2b = []
header_ome_fucci = []

for d in subfolders:
    
    ome_tifs = glob(path.join(d,'*.ome.tif'))
    ome_tifs = sorted(ome_tifs) # Sort by channel #
    ome_tifs = sorted(ome_tifs, key = sort_by_slice) # Sort by slice #
    if len(ome_tifs) < 40:
        print(f'Skipping {d}')
    else:
        if len(findall('1020nm',path.split(path.split(d)[0])[1])) == 0:
            header_ome_h2b.append(ome_tifs[0])
        else:
            header_ome_fucci.append(ome_tifs[0])

print(f'Found {len(header_ome_h2b)} B/G stacks and {len(header_ome_fucci)} R/R_shg stacks')

Found 1 B/G stacks and 1 R/R_shg stacks


# B/G: Load OME-TIFF and save *_reg.tif

In [37]:
import warnings
warnings.filterwarnings("ignore")

names = ['B','G']
registerZ = False; reference = 'B'

for header_ome in tqdm(header_ome_fucci):
    d = path.split(path.dirname(header_ome))[0]
    if np.all([path.exists(path.join(d,f'{chan}_reg.tif')) for chan in names]):
        print('All *_reg channels exist')
        continue
    
    # Load ome-tif
    print(f'Loading {d}')
    stack = io.imread(header_ome,is_ome=True)
    
    if stack.ndim > 3:
        images = [im for im in stack]
    else:
        images = [stack]

    if registerZ and reference is not None:
        # Use StackReg
        print(f'Registering {d}')
        sr = StackReg(StackReg.TRANSLATION) # There should only be slight sliding motion within a single stack
        T = sr.register_stack(B,reference='previous',n_frames=20,axis=0) #Obtain the transformation matrices
        for i,im in enumerate(images):
            images[i] = sr.transform_stack(im,tmats=T) # Apply to all channels

    for i,im in enumerate(images):
        output_path = path.join( d,f'{names[i]}_reg.tif')
        io.imsave(output_path,util.img_as_uint(im/im.max()),check_contrast=False)
    
    print(f'Saved with {output_path}')

100%|██████████████████████████████████████████████| 1/1 [00:00<00:00, 2212.19it/s]

All *_reg channels exist


# R/R_shg: Load OME-TIFF and save *_reg.tif

2

In [33]:
import warnings
warnings.filterwarnings("ignore")

names = ['R','R_shg']
registerZ = False; reference = 'R'

for header_ome in tqdm(header_ome_fucci):
    d = path.split(path.dirname(header_ome))[0]
    if np.all([path.exists(path.join(d,f'{chan}_reg.tif')) for chan in names]):
        print('All *_reg channels exist')
        continue 
    
    # Load ome-tif
    print(f'Loading {d}')
    stack = io.imread(header_ome,is_ome=True)
    
    if stack.ndim > 3:
        images = [im for im in stack]
    else:
        images = [stack]

    if registerZ and reference is not None:
        # Use StackReg
        print(f'Registering {d}')
        sr = StackReg(StackReg.TRANSLATION) # There should only be slight sliding motion within a single stack
        T = sr.register_stack(B,reference='previous',n_frames=20,axis=0) #Obtain the transformation matrices
        for i,im in enumerate(images):
            images[i] = sr.transform_stack(im,tmats=T) # Apply to all channels

    for i,im in tqdm(enumerate(images)):
        output_path = path.join( d,f'{names[i]}_reg.tif')
        io.imsave(output_path,util.img_as_uint(im/im.max()),check_contrast=False)
        
        print(f'Saved with {output_path}')

100%|██████████████████████████████████████████████| 1/1 [00:00<00:00, 2160.90it/s]

All *_reg channels exist


In [17]:
stackview.switch({names[i]:images[i] for i in range(len(names))},
                colormap=['pure_red','pure_green','pure_blue'],
                toggleable=True,
                zoom_factor=0.5)
